In [1]:
def load_annotated_stimulus_presentations_for_behavior_session(behavior_session_id, analysis_files_root_dir): 
    '''
    Load annotated_stimulus_presentations tables for one behavior session
    analysis_files_root_dir should be the directory that contains the "visual_behavior_analysis_files" data asset
    '''
    filepath = str(behavior_session_id)+'_annotated_stimulus_presentations'
    save_path = os.path.join(get_annotated_stimulus_presentations_save_dir(analysis_files_root_dir), filepath+'.h5')
    annotated_stimulus_presentations = pd.read_hdf(save_path, key='df')
    return annotated_stimulus_presentations


def load_annotated_stimulus_presentations_for_behavior_sessions(behavior_session_ids, analysis_files_root_dir): 
    '''
    Load and concatenate annotated_stimulus_presentations tables for a list of behavior_session_ids
    annotated_stimulus_presentations for different behavior sessions can be separated using the `behavior_session_id` column
    annotated_stimulus_presentations can be merged with stimulus_response_df tables by merging with the `ophys_experiment_table` on `behavior_session_id`, 
    then into `stimulus_response_df` on 'ophys_experiment_id`
    analysis_files_root_dir should be the directory that contains the "visual_behavior_analysis_files" data asset
    '''
    concatenated_annotated_stimulus_presentations = pd.DataFrame()
    for behavior_session_id in behavior_session_ids: 
        filepath = str(behavior_session_id)+'_annotated_stimulus_presentations'
        save_path = os.path.join(get_annotated_stimulus_presentations_save_dir(analysis_files_root_dir), filepath+'.h5')
        annotated_stimulus_presentations = pd.read_hdf(save_path, key='df')
        concatenated_annotated_stimulus_presentations = pd.concat([concatenated_annotated_stimulus_presentations, annotated_stimulus_presentations])
    return concatenated_annotated_stimulus_presentations

In [2]:
# We need to import these modules to get started
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# seaborn makes pretty plots & sets font sizes nicely
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})

# magic functions for jupyter notebook plotting
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# confirm that you are currently using the newest version of SDK (2.16.2)
import allensdk

In [4]:
import platform
platstring = platform.platform()

if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2024/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2024/"

In [5]:
# import behavior projet cache class from SDK to be able to load the data
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

cache = VisualBehaviorOphysProjectCache.from_local_cache(cache_dir=data_root, use_static_cache=True)
# if we needed to download the data we could have used the following line
# cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_root)  

# Get table describing ophys experiments *MOST USEFUL TABLE TO LOOK AT
ophys_experiments = cache.get_ophys_experiment_table()  

print('Number of ophys experiments = {}'.format(len(ophys_experiments)))

# get metadata tables
behavior_sessions = cache.get_behavior_session_table()


ophys_sessions = cache.get_ophys_session_table()


#ophys_cells = cache.get_ophys_cells_table()

#print number of items in each table for all imaging and behavioral sessions
#print('Number of behavior sessions = {}'.format(len(behavior_sessions)))
#print('Number of ophys sessions = {}'.format(len(ophys_sessions)))

import brain_observatory_utilities.datasets.optical_physiology.data_formatting as data_formatting

import brain_observatory_utilities.datasets.behavior.data_formatting as behavior_data_formatting

import pickle as pickle

Number of ophys experiments = 1936


/opt/conda/lib/python3.10/site-packages/allensdk/brain_observatory/behavior/behavior_project_cache/behavior_project_cache.py:135: UpdatedStimulusPresentationTableWarning: 
	As of AllenSDK version 2.16.0, the latest Visual Behavior Ophys data has been significantly updated from previous releases. Specifically the user will need to update all processing of the stimulus_presentations tables. These tables now include multiple stimulus types delineated by the columns `stimulus_block` and `stimulus_block_name`.

The data that was available in previous releases are stored in the block name containing 'change_detection' and can be accessed in the pandas table by using: 
	`stimulus_presentations[stimulus_presentations.stimulus_block_name.str.contains('change_detection')]`
  warnings.warn(


In [6]:
# Build desired events/dff/behavior table for a single animal

def get_stacked_change_response(ophys_experiment_id):
    # Load the data for this ophys experiment from the cache
    ophys_experiment = cache.get_behavior_ophys_experiment(ophys_experiment_id)

    # Get stimulus presentations
    #stimulus_presentations = ophys_experiment.stimulus_presentations.copy()
    
    
    stimulus_presentations = behavior_data_formatting.get_annotated_stimulus_presentations(ophys_experiment, epoch_duration_mins=5)


    # Add a column for whether the previous image was novel
    stimulus_presentations['prev_image_novel'] = stimulus_presentations['is_image_novel'].shift(1)
    
    # Save a subset of the columns in stimulus presentations
    stim_presentations_subset = stimulus_presentations[['trials_id','stimulus_block_name', 'image_index', 'image_name', 'duration', 
                                                        'start_time', 'end_time','start_frame', 'end_frame', 'is_change', 'is_image_novel', 
                                                        'prev_image_novel', 'omitted', 'flashes_since_change', 'is_sham_change', 'active',
                                                        'stimulus_name', 'licks', 'change_trials_id', 'change_time', 'go', 'catch', 
                                                        'auto_rewarded', 'hit', 'miss', 'false_alarm', 'correct_reject', 'response_time',
                                                        'reward_time', 'reward_volume', 'response_latency', 'time_from_last_change', 'licked', 
                                                        'stimulus_count_within_trial', 'previous_image_name', 'previous_response_on_trial',
                                                        'previous_change_on_trial', 'could_change', 'pre_change', 'lick_on_next_flash', 
                                                        'pre_omitted', 'post_omitted', 'reward_rate','engaged', 'engagement_state', 'epoch']]
    
    
    
    

    # Get stimulus aligned responses for all cells in this dataset: dFF
    stim_response_dff = data_formatting.get_stimulus_response_df(ophys_experiment, data_type='dff', event_type='changes',
                                                            time_window=[-1, 0.75], response_window_duration=0.25,
                                                            interpolate=True, output_sampling_rate=30)

    # Get stimulus aligned responses for all cells in this dataset: events
    # stim_response_events = data_formatting.get_stimulus_response_df(ophys_experiment, data_type='events', event_type='changes',
    #                                                         time_window=[-1, 0.75], response_window_duration=0.25,
    #                                                         interpolate=True, output_sampling_rate=10)
    
    # Get stimulus aligned responses for all cells in this dataset: filtered events
    stim_response_filtered_events = data_formatting.get_stimulus_response_df(ophys_experiment, data_type='filtered_events', event_type='changes',
                                                            time_window=[-1, 0.75], response_window_duration=0.25,
                                                            interpolate=True, output_sampling_rate=10)
    
    # Get stimulus aligned responses for all cells in this dataset: running speed
    stim_response_running_speed = data_formatting.get_stimulus_response_df(ophys_experiment, data_type='running_speed', event_type='changes',
                                                            time_window=[-1, 0.75], response_window_duration=0.25,
                                                            interpolate=True, output_sampling_rate=30)
    
    # Get stimulus aligned responses for all cells in this dataset: pupil_diameter
    stim_response_pupil_diameter = data_formatting.get_stimulus_response_df(ophys_experiment, data_type='pupil_diameter', event_type='changes',
                                                            time_window=[-1, 0.75], response_window_duration=0.25,
                                                            interpolate=True, output_sampling_rate=30)
    
    
    # Get stimulus aligned responses for all cells in this dataset: lick rate
    stim_response_lick_rate = data_formatting.get_stimulus_response_df(ophys_experiment, data_type='lick_rate', event_type='changes',
                                                            time_window=[-1, 0.75], response_window_duration=0.25,
                                                            interpolate=True, output_sampling_rate=30)
    
 



    # Rename the trace columns to be specific to dff
    stim_response_dff.rename(columns={"trace": "dff_trace", "mean_response": "dff_mean_response", 
                                  "baseline_response": "dff_baseline_response"}, inplace=True)



    # Add the events trace columns to the stim_response_dff data frame
    # stim_response_dff['events_trace'] = stim_response_events['trace']
    # stim_response_dff['events_mean_response'] = stim_response_events['mean_response']
    # stim_response_dff['events_baseline_response'] = stim_response_events['baseline_response']
    
    
    
    # Add the filtered events trace columns to the stim_response_dff data frame
    stim_response_dff['filtered_events_trace'] = stim_response_filtered_events['trace']
    stim_response_dff['filtered_events_mean_response'] = stim_response_filtered_events['mean_response']
    stim_response_dff['filtered_events_baseline_response'] = stim_response_filtered_events['baseline_response']
    stim_response_dff['filtered_events_trace_timestamps'] = stim_response_filtered_events['trace_timestamps']
    
    
    # Add the running_speed trace columns to the stim_response_dff data frame
    stim_response_dff['running_speed_trace'] = stim_response_running_speed['trace']
    stim_response_dff['running_speed_mean_response'] = stim_response_running_speed['mean_response']
    stim_response_dff['running_speed_baseline_response'] = stim_response_running_speed['baseline_response']
    
    
    # Add the pupil_diameter trace columns to the stim_response_dff data frame
    stim_response_dff['pupil_diameter_trace'] = stim_response_pupil_diameter['trace']
    stim_response_dff['pupil_diameter_mean_response'] = stim_response_pupil_diameter['mean_response']
    stim_response_dff['pupil_diameter_baseline_response'] = stim_response_pupil_diameter['baseline_response']
    
    
    # Add the lick_rate trace columns to the stim_response_dff data frame
    stim_response_dff['lick_rate_trace'] = stim_response_lick_rate['trace']
    stim_response_dff['lick_rate_mean_response'] = stim_response_lick_rate['mean_response']
    stim_response_dff['lick_rate_baseline_response'] = stim_response_lick_rate['baseline_response']
    
    
    
    
    


    # Merge the stimulus response dataframe with the stimulus table to get information about what stimuli were presented 
    stim_response_stack = stim_response_dff.merge(stim_presentations_subset, on='stimulus_presentations_id')



    stim_response_stack['ophys_experiment_id'] = ophys_experiment_id

    

    return stim_response_stack
    

In [7]:
ophys_experiments = ophys_experiments.sort_values(by=['cre_line', 'mouse_id', 'date_of_acquisition'])
#ophys_experiments[['mouse_id', 'cre_line', 'sex', 'date_of_acquisition', 'session_type', 'session_number', 'prior_exposures_to_image_set', 'prior_exposures_to_session_type']]

In [8]:
# Get only the OPHYS_4 experiments with no prior exposure to the novel image set
subset_ophys_experiments = ophys_experiments[(ophys_experiments.project_code=='VisualBehaviorMultiscope4areasx2d')&
                        (ophys_experiments.session_number==4) &
                        (ophys_experiments.prior_exposures_to_image_set==0)]

# Sort the filtered experiments by genotype, sex, mouse ID
subset_ophys_experiments = subset_ophys_experiments.sort_values(by=['cre_line', 'targeted_structure', 'sex', 'mouse_id'])


#subset_ophys_experiments[['cre_line', 'targeted_structure', 'sex', 'mouse_id', 'session_type', 'prior_exposures_to_image_set', 'prior_exposures_to_session_type']]

# there are 4 Slc17a7 animals, 4 Sst animals, and 4 Vip animals

In [9]:
subset_ophys_experiments.columns

Index(['behavior_session_id', 'ophys_session_id', 'ophys_container_id',
       'mouse_id', 'indicator', 'full_genotype', 'driver_line', 'cre_line',
       'reporter_line', 'sex', 'age_in_days', 'imaging_depth',
       'targeted_structure', 'targeted_imaging_depth', 'imaging_plane_group',
       'project_code', 'session_type', 'session_number', 'image_set',
       'behavior_type', 'passive', 'experience_level',
       'prior_exposures_to_session_type', 'prior_exposures_to_image_set',
       'prior_exposures_to_omissions', 'date_of_acquisition', 'equipment_name',
       'published_at', 'isi_experiment_id', 'file_id'],
      dtype='object')

In [10]:
# Get some set of behavior sessions
selected_behavior_sessions = behavior_sessions[(behavior_sessions.project_code=='VisualBehaviorMultiscope4areasx2d')&
                                                (behavior_sessions.session_number==4) &
                                                (behavior_sessions.prior_exposures_to_image_set==0)]

In [11]:
selected_behavior_sessions

,ophys_session_id,ophys_container_id,mouse_id,indicator,full_genotype,driver_line,cre_line,reporter_line,sex,age_in_days,...,num_targeted_structures,engaged_trial_count,go_trial_count,miss_trial_count,hit_trial_count,false_alarm_trial_count,correct_reject_trial_count,trial_count,catch_trial_count,file_id
behavior_session_id,,,,,,,,,,,,,,,,,,,,,
1010371498,1010346697,"[1018027796, 1018027799, 1018027802, 101802780...",499478,GCaMP6f,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Vip-IRES-Cre],Vip-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,142,...,4,313,289,156,133,0,42,533,42,-999
977451727,977288921,"[1018027747, 1018027750, 1018027753, 101802775...",484408,GCaMP6f,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,"[Slc17a7-IRES2-Cre, Camk2a-tTA]",Slc17a7-IRES2-Cre,Ai93(TITL-GCaMP6f),M,121,...,4,364,290,157,133,1,41,537,42,-999
993957374,993942673,"[1018028254, 1018028261, 1018028267, 101802827...",485689,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),M,151,...,4,419,219,124,95,2,30,707,32,-999
1102183212,1102166150,"[1099411349, 1099411355, 1099411361, 111274486...",555970,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),M,217,...,4,19,342,337,5,1,46,457,47,-999
993817409,993790878,"[1022733783, 1022733786, 1022733792, 102273379...",484631,GCaMP6f,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,"[Slc17a7-IRES2-Cre, Camk2a-tTA]",Slc17a7-IRES2-Cre,Ai93(TITL-GCaMP6f),F,156,...,4,229,242,131,111,3,31,863,34,-999
966754037,<NA>,NaN,477052,GCaMP6f,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,"[Slc17a7-IRES2-Cre, Camk2a-tTA]",Slc17a7-IRES2-Cre,Ai93(TITL-GCaMP6f),M,135,...,<NA>,219,165,61,104,9,15,661,24,3453
1098360551,1098326266,"[1095625228, 1095625231, 1095625235, 109562523...",554115,GCaMP6f,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Vip-IRES-Cre],Vip-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),M,213,...,4,207,167,83,84,7,17,844,24,-999
1109864540,1109840611,"[1107904115, 1115895066, 1115959856, 111595986...",570949,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,139,...,4,44,338,312,26,1,47,440,48,-999
970691928,<NA>,NaN,481295,GCaMP6f,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,"[Slc17a7-IRES2-Cre, Camk2a-tTA]",Slc17a7-IRES2-Cre,Ai93(TITL-GCaMP6f),F,121,...,<NA>,328,307,171,136,2,42,500,44,4037


In [12]:
behavior_session_ids = selected_behavior_sessions.index.unique()
behavior_session_ids

Int64Index([1010371498,  977451727,  993957374, 1102183212,  993817409,
             966754037, 1098360551, 1109864540,  970691928, 1106017102,
            1101451349, 1059672853,  962242716, 1039520735, 1106065301,
            1089052619, 1010487593],
           dtype='int64', name='behavior_session_id')

In [14]:
analysis_files_root_dir = '/root/capsule/scratch/uploads/'


In [15]:
load_annotated_stimulus_presentations_for_behavior_sessions(behavior_session_ids, analysis_files_root_dir) 

NameError: name 'get_annotated_stimulus_presentations_save_dir' is not defined